https://www.psycopg.org/docs/

https://pythonru.com/biblioteki/vvedenie-v-postgresql-s-python-psycopg2

https://docs.python.org/3/library/zipfile.html

In [27]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import psycopg2

In [28]:
# проверка доступа к БД
conn = psycopg2.connect("""
    host=rc1b-itt1uqz8cxhs0c3d.mdb.yandexcloud.net
    port=6432
    sslmode=verify-full
    dbname=market_db
    user=sfedyusnin
    password=Qazwsx123Qaz
    target_session_attrs=read-write
""")

q = conn.cursor()
q.execute('SELECT version()')

print(q.fetchone())

conn.close()

('PostgreSQL 14.1 (Ubuntu 14.1-201-yandex.52142.672784f35a) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',)


In [29]:
path = r'./data/'

In [30]:
# формируем список имен файлов отчетов по кампаниям
camp_files = []
all_files = glob.glob(os.path.join(path, "*.*"))
for file in all_files:
    if os.path.basename(file).startswith('camp'):
        print(file)
        with zipfile.ZipFile(file) as zf:
            camp_files+=zf.namelist()
    
# распаковываем архивы
zip_files = glob.glob(os.path.join(path, "*.zip"))
for file in zip_files:
    print(file)
    with zipfile.ZipFile(file) as zf:
        zf.extractall(path)

./data\campaigns_0.zip
./data\campaigns_1.zip
./data\campaigns_0.zip
./data\campaigns_1.zip


In [31]:
camp_files

['835460_10.05.2022-28.06.2022.csv',
 '835659_10.05.2022-28.06.2022.csv',
 '941148_10.05.2022-28.06.2022.csv',
 '1070352_10.05.2022-28.06.2022.csv',
 '1092333_10.05.2022-28.06.2022.csv',
 '1168542_10.05.2022-28.06.2022.csv',
 '781143_10.05.2022-28.06.2022.csv',
 '820263_10.05.2022-28.06.2022.csv',
 '916272_10.05.2022-28.06.2022.csv',
 '916273_10.05.2022-28.06.2022.csv',
 '376753_10.05.2022-28.06.2022.csv',
 '380229_10.05.2022-28.06.2022.csv',
 '449864_10.05.2022-28.06.2022.csv',
 '467037_10.05.2022-28.06.2022.csv',
 '370873_10.05.2022-28.06.2022.csv',
 '370877_10.05.2022-28.06.2022.csv',
 '370884_10.05.2022-28.06.2022.csv',
 '376752_10.05.2022-28.06.2022.csv']

In [32]:
# функция для обработки датасетов отчетов по кампаниям
def camp_read_trans(path):
    data = pd.read_csv(path, sep=';')
    data = data.reset_index()
    
    camp = data.keys()[-1].split(',')[0].split()[-1]
    period = data.keys()[-1].split(',')[1].split()[-1]
    
    data.columns = data[0:1].values.tolist()[0]
    data.drop(index=0, inplace=True)
    data.drop(data.tail(1).index, inplace=True)
    
    data['Кампания'] = camp
    data['Период'] = period
    
    data = data[data.columns[-2:].tolist()+data.columns[:-2].tolist()]
    data = data[data.columns.dropna()]
    data = data.dropna(axis=0, how='any', thresh=10)
    return(data)

In [33]:
camp_data = []
for file in camp_files:
    camp_data.append(camp_read_trans(path+file))

In [34]:
camp_dataset = pd.concat(camp_data, axis=0).reset_index().drop('index', axis=1)

In [35]:
camp_dataset

,Кампания,Период,Тип условия,Условие показа,Платформа,Показы,Клики,CTR (%),Охват,Средняя ставка за 1000 показов (руб.),...,Ozon ID,Ozon ID рекламируемого товара,Артикул,Наименование,Количество,Цена продажи,"Стоимость, руб.","Ставка, %","Ставка, руб.","Расход, руб."
0,835659,10.05.2022-28.06.2022,NaN,NaN,NaN,247,4,"1,62",NaN,"300,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,835659,10.05.2022-28.06.2022,NaN,Радио- и видеоняни,NaN,6882,32,"0,46",NaN,"300,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,835659,10.05.2022-28.06.2022,NaN,NaN,NaN,3,0,"0,00",NaN,"300,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,835659,10.05.2022-28.06.2022,NaN,Радио- и видеоняни,NaN,14280,40,"0,28",NaN,"300,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,835659,10.05.2022-28.06.2022,NaN,NaN,NaN,64,0,"0,00",NaN,"0,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,370884,10.05.2022-28.06.2022,NaN,уличное видеонаблюдение,NaN,1,0,"0,00",NaN,"400,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
747,370884,10.05.2022-28.06.2022,NaN,уличное видеонаблюдение комплект,NaN,1,0,"0,00",NaN,"400,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
748,370884,10.05.2022-28.06.2022,NaN,умная камера,NaN,57,0,"0,00",NaN,"428,07",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
749,370884,10.05.2022-28.06.2022,NaN,цифровая камера,NaN,9,0,"0,00",NaN,"317,00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
camp_dataset.columns

Index(['Кампания', 'Период', 'Тип условия', 'Условие показа', 'Платформа',
       'Показы', 'Клики', 'CTR (%)', 'Охват',
       'Средняя ставка за 1000 показов (руб.)', 'Расход (руб., с НДС)', 'sku',
       'Название товара', 'Цена товара (руб.)', 'Тип страницы', 'Заказы',
       'Выручка (руб.)', 'Заказы модели', 'Выручка с заказов модели (руб.)',
       'Баннер', 'Средняя ставка (руб.)', 'Дата', 'ID заказа', 'Номер заказа',
       'Ozon ID', 'Ozon ID рекламируемого товара', 'Артикул', 'Наименование',
       'Количество', 'Цена продажи', 'Стоимость, руб.', 'Ставка, %',
       'Ставка, руб.', 'Расход, руб.'],
      dtype='object')

In [37]:
# формируем список имен файлов отчетов по media, product
mp_files = []
csv_files = glob.glob(os.path.join(path, "*.csv"))
for file in csv_files:
    if os.path.basename(file).startswith('media'):
        mp_files.append(file)
    elif os.path.basename(file).startswith('product'):
        mp_files.append(file)

In [38]:
mp_files

['./data\\media_2022-05-10-2022-06-28.csv',
 './data\\product_2022-05-10-2022-06-28.csv']

In [39]:
def mp_read_trans(path):
    data = pd.read_csv(path, sep=';')    
    period = path.split('_')[1].split('.')[0]   
    data['Период'] = period
    return(data)  

In [40]:
mp_data = []
for file in mp_files:
    mp_data.append(mp_read_trans(file))

In [41]:
mp_data[0].shape

(8, 18)

In [42]:
mp_data[1].shape

(6, 15)

In [43]:
mp_dataset = pd.concat(mp_data, axis=0).reset_index().drop('index', axis=1)

In [44]:
mp_dataset

,ID,Название,Формат,Статус,"Дневной бюджет, ₽","Бюджет, ₽",Приоритет,"Расход, ₽",Показы,Клики,"Средняя ставка, ₽","Ср. цена 1000 показов, ₽","CTR, %","Ср. цена клика, ₽","Заказы, шт.","Заказы, руб",ДРР,Период
0,376752,Брендовая полка,BRAND_SHELF,inactive,1000,0.0,auction,0.000,0,1,0.000000,0.000000,"0,00",0.000000,1,790,"0,00",2022-05-10-2022-06-28
1,376753,Баннер,BANNER,inactive,1000,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",2022-05-10-2022-06-28
2,380229,Баннер,BANNER,inactive,500,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",2022-05-10-2022-06-28
3,820263,Брендовая полка 1.2,BRAND_SHELF,running,1000,0.0,auction,17187.400,85937,2754,0.200000,200.000000,"0,03",6.240886,4,23146,"74,26",2022-05-10-2022-06-28
4,835460,Брендовая полка 2,BRAND_SHELF,inactive,500,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",2022-05-10-2022-06-28
5,941148,Брендовая полка,BRAND_SHELF,inactive,500,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",2022-05-10-2022-06-28
6,1070352,Брендовая полка 1,BRAND_SHELF,running,1000,0.0,auction,18561.400,92807,2915,0.200000,200.000000,"0,03",6.367547,4,23489,"79,02",2022-05-10-2022-06-28
7,1092333,Сегмент для подвижения,BANNER,inactive,500,0.0,auction,0.000,0,0,0.000000,0.000000,"0,00",0.000000,0,0,"0,00",2022-05-10-2022-06-28
8,370877,Карточка товара видеокамеры,NaN,running,1000,NaN,NaN,7396.900,26896,256,0.239768,275.018590,"0,01",28.894141,6,36450,"20,29",2022-05-10-2022-06-28
9,370884,Поиск купольная и буллет,NaN,running,1000,NaN,NaN,35694.385,74719,1343,0.489241,477.714972,"0,02",26.578098,12,81421,"43,84",2022-05-10-2022-06-28


In [45]:
mp_dataset.columns

Index(['ID', 'Название', 'Формат', 'Статус', 'Дневной бюджет, ₽', 'Бюджет, ₽',
       'Приоритет', 'Расход, ₽', 'Показы', 'Клики', 'Средняя ставка, ₽',
       'Ср. цена 1000 показов, ₽', 'CTR, %', 'Ср. цена клика, ₽',
       'Заказы, шт.', 'Заказы, руб', 'ДРР', 'Период'],
      dtype='object')

In [46]:
# формируем список имен файлов отчетов daily
d_files = []
csv_files = glob.glob(os.path.join(path, "*.csv"))
for file in csv_files:
    if os.path.basename(file).startswith('daily'):
        d_files.append(file)

In [47]:
d_files

['./data\\daily_2022-05-10-2022-06-28.csv',
 './data\\daily_2022-06-26-2022-06-28.csv']

In [48]:
d_data = []
for file in d_files:
    d_data.append(pd.read_csv(file, sep=';'))

In [49]:
d_data[0].shape

(432, 9)

In [50]:
d_data[1].shape

(27, 9)

In [51]:
d_dataset = pd.concat(d_data, axis=0).reset_index().drop('index', axis=1)
d_dataset = d_dataset.drop_duplicates()

In [52]:
d_dataset

,ID,Название,Дата,Показы,Клики,"Расход, ₽","Средняя ставка, ₽","Заказы, шт.","Заказы, руб"
0,370877,Карточка товара видеокамеры,2022-05-10,1271,13,"500,60","0,39",1,"5390,00"
1,370877,Карточка товара видеокамеры,2022-05-11,1287,10,"504,30","0,39",0,"0,00"
2,370877,Карточка товара видеокамеры,2022-05-12,1279,9,"501,70","0,39",0,"0,00"
3,370877,Карточка товара видеокамеры,2022-05-13,1286,9,"503,60","0,39",0,"0,00"
4,370877,Карточка товара видеокамеры,2022-05-14,1285,11,"504,40","0,39",0,"0,00"
...,...,...,...,...,...,...,...,...,...
427,820263,Брендовая полка 1.2,2022-06-24,1890,80,"378,00","0,20",0,"0,00"
428,820263,Брендовая полка 1.2,2022-06-25,1656,107,"331,20","0,20",0,"0,00"
429,820263,Брендовая полка 1.2,2022-06-26,1808,47,"361,60","0,20",0,"0,00"
430,820263,Брендовая полка 1.2,2022-06-27,2213,65,"442,60","0,20",0,"0,00"
